<a href="https://colab.research.google.com/github/girupashankar/Data_Analysis_Projects_Using_Python/blob/main/Spotify_Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify Music Recommendation System using Python

I hope you have understood what a Music Recommendation System is. Now, in this section, I'll take yoou through building a Music Reommendation System using Spotify API and Python.  
To get started with building a Music Recommendation System, we first need to have an access token. The access token serves as a temporary authorization credential, allowing the code to make authenticated requests to the Spotify API on behalf of the application. Below is how we can get it:

In [2]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '57412ee23b454be9b79ff730c33cc8d8'
CLIENT_SECRET = '5679138d85a84beeb7230d16e5528604'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}',
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {
    'grant_type' : 'client_credentials'
}
response = requests.post(token_url, headers=headers, data=data)

if response.status_code == 200:
  # Extract the access token from the response
  access_token = response.json()['access_token']
  print("Access Token:", access_token, "obtained successfully.")
else:
  print("Error:", response.text)

Access Token: BQD2Jh5pqamDt30E82qIMyw-ryUmAi9JtfI2l9WtDwy4U8yOj1ARtmtiBmof7Cc_RfLQpz3swGVbEMuFTbaLQVUxXcjzM1h4ziwqh4Lh1cw4PVOzVQo obtained successfully.


In the above code, The CLIENT_ID and CLIENT_SECRET variables hold my credentials (you need to add your credentials in these variables) that uniquely identify the application making requests to the Spotify API.  
These credentials are obtained when a developer registers their application with Spotify’s developer dashboard. The Client ID identifies the application, while the Client Secret is a confidential key used for authentication.  
The client ID and secret are combined in the client_credentials variable, separated by a colon (:). Then, this string is encoded using Base64 encoding to create a secure representation of the credentials. We then proceed to request an access token from the Spotify API.

It sends a POST request to the token_url (https://accounts.spotify.com/api/token) with the client credentials in the Authorization header, which is required for client authentication. The grant_type parameter is set to ‘client_credentials’ to indicate that the application is requesting an access token for the client credentials flow.

With the access token, the application can now make authorized requests to retrieve music data, such as tracks, albums, artists, and user information, which is fundamental for building a music recommendation system using the Spotify API and Python.


Now, I’ll write a function to get music data from any playlist on Spotify. For this task, you need to install the Spotipy library, which is a Python library providing access to Spotify’s web API. Here’s how to install it on your system by writing the command mentioned below in your command prompt or terminal:

* pip install spotipy  

Below I am defining a function responsible for collecting music data from any playlist on Spotify using the Spotipy library:

In [5]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 5.6 MB/s eta 0:00:00


## Step 1: Data Acqusition

In [6]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [7]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

The function begins by initializing the Spotipy client with the provided access_token, which serves as the authentication token to interact with the Spotify Web API. The access_token allows the function to make authorized requests to access Spotify’s resources. The function then uses the Spotipy client to fetch information about the tracks in the specified playlist (identified by its playlist_id). The sp.playlist_tracks method retrieves the playlist tracks. The fields parameter is used to specify the specific track information that is required, such as track ID, name, artists, album ID, and album name.

The function then extracts relevant information from the retrieved playlist tracks and stores it in a list of dictionaries called music_data. For each track in the playlist, the function extracts data such as track name, artists (combined into a single string), album name, album ID, track ID, and popularity. The function uses the sp.audio_features method to fetch audio features for each track in the playlist. These audio features include attributes like danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, etc. These audio features provide insights into the characteristics of each track.


The extracted information for all tracks is stored in the music_data list. The function then creates a DataFrame from the music_data list. The DataFrame organizes the music data in a tabular format, making it easier to analyze and work with the collected information.

Now, here’s how we can use the function to collect music data from any playlist on Spotify:

In [60]:
playlist_id = '37i9dQZF1DX4Im4BTs2WMg'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,"Achacho - From ""Aranmanai 4""","Hiphop Tamizha, Kharesma Ravichandran, Srinish...","Achacho (From ""Aranmanai 4"")",38v7imWGztnKKx1SZuf5o3,5eSoySxUnx3xsm0FHFgHiv,70,2024-04-14,193056,False,https://open.spotify.com/track/5eSoySxUnx3xsm0...,...,0.788,9,-3.940,0,0.2280,0.1100,0.000000,0.1380,0.845,127.917
1,Kadharalz,Anirudh Ravichander,Indian 2 (Original Motion Picture Soundtrack),2odSkyvlROPRtPYsr2lE0l,77OQsnAirm7Ke8EGgBeOxz,53,2024-06-01,234023,False,https://open.spotify.com/track/77OQsnAirm7Ke8E...,...,0.902,1,-5.411,1,0.2980,0.0638,0.004660,0.5390,0.735,160.025
2,Illuminati,"Sushin Shyam, Dabzee, Vinayak Sasikumar",Aavesham (Original Motion Picture Soundtrack),3E0V1VegCFH9SeRSNWFKki,1a1xLj9W8libnO9PvJf6ao,72,2024-04-19,212525,False,https://open.spotify.com/track/1a1xLj9W8libnO9...,...,0.817,11,-5.940,0,0.1240,0.2320,0.000003,0.2150,0.553,138.002
3,Come Back Indian,Anirudh Ravichander,Indian 2 (Original Motion Picture Soundtrack),2odSkyvlROPRtPYsr2lE0l,3Be1gnCOJX50ZSg8iHgIDv,51,2024-06-01,217076,False,https://open.spotify.com/track/3Be1gnCOJX50ZSg...,...,0.905,11,-5.672,0,0.1320,0.0335,0.000000,0.0687,0.357,114.980
4,"Katchi Sera - From ""Think Indie""",Sai Abhyankkar,"Katchi Sera (From ""Think Indie"")",1kqsloJJsGKKKiWhinel6F,1zzejMGRYKP5XOa3FmzXfa,74,2024-01-22,181818,False,https://open.spotify.com/track/1zzejMGRYKP5XOa...,...,0.642,7,-8.480,0,0.1150,0.2430,0.001040,0.3390,0.822,148.465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,"Porkanda Singam (EDM Version) [From ""Vikram""]",Anirudh Ravichander,"Porkanda Singam (EDM Version) [From ""Vikram""]",4GU1vqDAiySfL0K3sXLpT2,12LE4Omy9ze8xTOMofMVB3,61,2022-06-24,149434,False,https://open.spotify.com/track/12LE4Omy9ze8xTO...,...,0.755,11,-6.045,0,0.0476,0.1020,0.194000,0.1590,0.422,158.932
71,"Jalabulajangu (From ""Don"")","Anirudh Ravichander, Rokesh","Jalabulajangu (From ""Don"")",3cXSg3K9z6fZAd1lWmqvrc,4HdzYTQiouB9BVYX7JOrhh,65,2021-12-16,202608,False,https://open.spotify.com/track/4HdzYTQiouB9BVY...,...,0.823,7,-5.437,0,0.1530,0.0261,0.000000,0.0481,0.457,151.047
72,"Bodhai Kaname - From ""Oh Manapenne""","Vishal Chandrashekhar, Anirudh Ravichander, Sh...","Bodhai Kaname (From ""Oh Manapenne"")",53xK2qW53doSVnfWafunKQ,4qIMmEjV8gaEsfZML9LALx,63,2021-07-30,261948,False,https://open.spotify.com/track/4qIMmEjV8gaEsfZ...,...,0.405,7,-12.327,1,0.0424,0.1520,0.000003,0.1540,0.267,160.047
73,Otha Thamarai - Original Soundtrack,"Bala, Nixen, Sandy Sandellow",Otha Thamarai (Original Soundtrack),6sdalaw3mTPzUomDNnWjeN,7e97QgqoYTXOj1s7JI0dHR,53,2022-04-29,219923,False,https://open.spotify.com/track/7e97QgqoYTXOj1s...,...,0.744,7,-6.191,0,0.2950,0.5070,0.000000,0.2600,0.877,194.976


In [61]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        75 non-null     object 
 1   Artists           75 non-null     object 
 2   Album Name        75 non-null     object 
 3   Album ID          75 non-null     object 
 4   Track ID          75 non-null     object 
 5   Popularity        75 non-null     int64  
 6   Release Date      75 non-null     object 
 7   Duration (ms)     75 non-null     int64  
 8   Explicit          75 non-null     bool   
 9   External URLs     75 non-null     object 
 10  Danceability      75 non-null     float64
 11  Energy            75 non-null     float64
 12  Key               75 non-null     int64  
 13  Loudness          75 non-null     float64
 14  Mode              75 non-null     int64  
 15  Speechiness       75 non-null     float64
 16  Acousticness      75 non-null     float64
 17 

In [62]:
music_df.shape

(75, 21)

In [63]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [64]:
music_df.describe()

,Popularity,Duration (ms),Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,59.280000,224792.746667,0.675067,0.702960,5.226667,-7.375813,0.480000,0.116128,0.300275,0.024026,0.167979,0.601093,122.750307
std,8.943607,65002.847535,0.129938,0.166411,3.581383,2.381687,0.502964,0.094053,0.242409,0.096142,0.142362,0.201005,27.800927
min,13.000000,47500.000000,0.324000,0.342000,0.000000,-13.552000,0.000000,0.027100,0.000690,0.000000,0.026200,0.167000,81.023000
25%,56.000000,195470.500000,0.590000,0.562500,2.000000,-9.070000,0.000000,0.042450,0.095800,0.000001,0.078050,0.457000,98.449000
50%,61.000000,228568.000000,0.671000,0.716000,6.000000,-6.630000,0.000000,0.078100,0.249000,0.000043,0.109000,0.587000,118.972000
75%,65.000000,266475.000000,0.765500,0.833500,8.500000,-5.627500,1.000000,0.160500,0.502000,0.004320,0.203500,0.762000,139.965000
max,74.000000,386009.000000,0.956000,0.980000,11.000000,-2.082000,1.000000,0.375000,0.875000,0.652000,0.671000,0.936000,194.976000


In [65]:
# Split the DataFrame into two DataFrames based on data type
categorical_df = music_df.select_dtypes(include=['object'])
numeric_df = music_df.select_dtypes(include=['int64', 'float64'])

In [66]:
print("Numeric Data:")
numeric_df

Numeric Data:


,Popularity,Duration (ms),Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,70,193056,0.841,0.788,9,-3.940,0,0.2280,0.1100,0.000000,0.1380,0.845,127.917
1,53,234023,0.671,0.902,1,-5.411,1,0.2980,0.0638,0.004660,0.5390,0.735,160.025
2,72,212525,0.783,0.817,11,-5.940,0,0.1240,0.2320,0.000003,0.2150,0.553,138.002
3,51,217076,0.786,0.905,11,-5.672,0,0.1320,0.0335,0.000000,0.0687,0.357,114.980
4,74,181818,0.765,0.642,7,-8.480,0,0.1150,0.2430,0.001040,0.3390,0.822,148.465
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,61,149434,0.685,0.755,11,-6.045,0,0.0476,0.1020,0.194000,0.1590,0.422,158.932
71,65,202608,0.635,0.823,7,-5.437,0,0.1530,0.0261,0.000000,0.0481,0.457,151.047
72,63,261948,0.516,0.405,7,-12.327,1,0.0424,0.1520,0.000003,0.1540,0.267,160.047
73,53,219923,0.541,0.744,7,-6.191,0,0.2950,0.5070,0.000000,0.2600,0.877,194.976


In [67]:
print("Categorical Data:")
categorical_df

Categorical Data:


,Track Name,Artists,Album Name,Album ID,Track ID,Release Date,External URLs
0,"Achacho - From ""Aranmanai 4""","Hiphop Tamizha, Kharesma Ravichandran, Srinish...","Achacho (From ""Aranmanai 4"")",38v7imWGztnKKx1SZuf5o3,5eSoySxUnx3xsm0FHFgHiv,2024-04-14,https://open.spotify.com/track/5eSoySxUnx3xsm0...
1,Kadharalz,Anirudh Ravichander,Indian 2 (Original Motion Picture Soundtrack),2odSkyvlROPRtPYsr2lE0l,77OQsnAirm7Ke8EGgBeOxz,2024-06-01,https://open.spotify.com/track/77OQsnAirm7Ke8E...
2,Illuminati,"Sushin Shyam, Dabzee, Vinayak Sasikumar",Aavesham (Original Motion Picture Soundtrack),3E0V1VegCFH9SeRSNWFKki,1a1xLj9W8libnO9PvJf6ao,2024-04-19,https://open.spotify.com/track/1a1xLj9W8libnO9...
3,Come Back Indian,Anirudh Ravichander,Indian 2 (Original Motion Picture Soundtrack),2odSkyvlROPRtPYsr2lE0l,3Be1gnCOJX50ZSg8iHgIDv,2024-06-01,https://open.spotify.com/track/3Be1gnCOJX50ZSg...
4,"Katchi Sera - From ""Think Indie""",Sai Abhyankkar,"Katchi Sera (From ""Think Indie"")",1kqsloJJsGKKKiWhinel6F,1zzejMGRYKP5XOa3FmzXfa,2024-01-22,https://open.spotify.com/track/1zzejMGRYKP5XOa...
...,...,...,...,...,...,...,...
70,"Porkanda Singam (EDM Version) [From ""Vikram""]",Anirudh Ravichander,"Porkanda Singam (EDM Version) [From ""Vikram""]",4GU1vqDAiySfL0K3sXLpT2,12LE4Omy9ze8xTOMofMVB3,2022-06-24,https://open.spotify.com/track/12LE4Omy9ze8xTO...
71,"Jalabulajangu (From ""Don"")","Anirudh Ravichander, Rokesh","Jalabulajangu (From ""Don"")",3cXSg3K9z6fZAd1lWmqvrc,4HdzYTQiouB9BVYX7JOrhh,2021-12-16,https://open.spotify.com/track/4HdzYTQiouB9BVY...
72,"Bodhai Kaname - From ""Oh Manapenne""","Vishal Chandrashekhar, Anirudh Ravichander, Sh...","Bodhai Kaname (From ""Oh Manapenne"")",53xK2qW53doSVnfWafunKQ,4qIMmEjV8gaEsfZML9LALx,2021-07-30,https://open.spotify.com/track/4qIMmEjV8gaEsfZ...
73,Otha Thamarai - Original Soundtrack,"Bala, Nixen, Sandy Sandellow",Otha Thamarai (Original Soundtrack),6sdalaw3mTPzUomDNnWjeN,7e97QgqoYTXOj1s7JI0dHR,2022-04-29,https://open.spotify.com/track/7e97QgqoYTXOj1s...


In [68]:
numeric_df.columns

Index(['Popularity', 'Duration (ms)', 'Danceability', 'Energy', 'Key',
       'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo'],
      dtype='object')

Now, let’s move further to building a music recommendation system using Python. Let’s import the necessary Python libraries now:

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

While providing music recommendations to users, it is important to recommend the latest releases. For this, we need to give more weight to the latest releases in the recommendations. Let’s write a function to solve this problem:

In [70]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

The above function takes the release date of a music track as input, which is provided in the format ‘YYYY-MM-DD’. It then uses the datetime.strptime function from the Python datetime module to convert the release date string to a datetime object. This conversion allows us to perform arithmetic operations with dates. The function then calculates the time span between the release date of the track and the current date (today’s date) using datetime.now() – release_date. This results in a timedelta object representing the time difference between the two dates.


The weighted popularity score is computed based on the time span. The formula to calculate the weight is 1 / (time_span.days + 1). The time_span.days attribute of the timedelta object gives the number of days in the time span between the release date and today. Adding 1 to the number of days ensures that the weight is never zero, even for very recent releases, as this would lead to a division by zero error.

The idea behind this formula is that the weight decreases as the time span between the release date and today increases. More recent releases will have a higher weight, while older releases will have a lower weight. As a result, when combining this weighted popularity score with other factors in a recommendation system, recent tracks will have a more significant impact on the final recommendations, reflecting users’ potential interest in newer music.

>Now let’s normalize the music features before moving forward:

In [71]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

We will create a hybrid recommendation system for music recommendations. The first approach will be based on recommending music based on music audio features, and the second approach will be based on recommending music based on weighted popularity.

>Here’s how to generate music recommendations based on the music audio features:

In [72]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [73]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Create a new DataFrame for the input song's data
    new_row = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    # Combine content-based recommendations and the new row using pd.concat
    hybrid_recommendations = pd.concat([content_based_rec, new_row], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [77]:
input_song_name = "Come Back Indian"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Come Back Indian':
                   Track Name  \
1                  Illuminati   
0   Kaavaalaa (From "Jailer")   
3  Jalabulajangu (From "Don")   
4     Adheeraa (From "Cobra")   
2               Yaakkai Thiri   

                                             Artists  \
1            Sushin Shyam, Dabzee, Vinayak Sasikumar   
0  Anirudh Ravichander, Shilpa Rao, Arunraja Kamaraj   
3                        Anirudh Ravichander, Rokesh   
4            A.R. Rahman, Vagu Mazan, Thoughtsfornow   
2          A.R. Rahman, Sunitha Sarathy, Pop Shalini   

                                          Album Name Release Date  Popularity  
1      Aavesham (Original Motion Picture Soundtrack)   2024-04-19        72.0  
0                          Kaavaalaa (From "Jailer")   2023-07-06        66.0  
3                         Jalabulajangu (From "Don")   2021-12-16        65.0  
4                            Adheeraa (From "Cobra")   2022-04-22        61.0  
2  Aayitha Ezhu

In [78]:
recommendations

,Track Name,Artists,Album Name,Release Date,Popularity
1,Illuminati,"Sushin Shyam, Dabzee, Vinayak Sasikumar",Aavesham (Original Motion Picture Soundtrack),2024-04-19,72.0
0,"Kaavaalaa (From ""Jailer"")","Anirudh Ravichander, Shilpa Rao, Arunraja Kamaraj","Kaavaalaa (From ""Jailer"")",2023-07-06,66.0
3,"Jalabulajangu (From ""Don"")","Anirudh Ravichander, Rokesh","Jalabulajangu (From ""Don"")",2021-12-16,65.0
4,"Adheeraa (From ""Cobra"")","A.R. Rahman, Vagu Mazan, Thoughtsfornow","Adheeraa (From ""Cobra"")",2022-04-22,61.0
2,Yaakkai Thiri,"A.R. Rahman, Sunitha Sarathy, Pop Shalini",Aayitha Ezhutu (Original Motion Picture Soundt...,2004-02-27,57.0
